<a href="https://colab.research.google.com/github/SrijaG29/offc_dsa_prac/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
<div class="main">
    <div class="navbar">
        <div class="location">
            <button>Find Location</button>
        </div>
        <div class="form" >
            <input type="text" placeholder="Address" id="address" [(ngModel)]="address">
            <input type="text" placeholder="City" id="city" [(ngModel)]="city">
            <input type="text" placeholder="Country" id="country" [(ngModel)]="country">
            <button type="submit" id="search" (click)="getALlPlaces()">Search</button>
        </div>
    </div>
    <div class="mainbody" *ngIf="mainBody">
        <div class="locationdetails">
          <p><b>{{name}}</b></p>
          <p>{{addresstype}}<p>
          <p>{{displayname}}</p>
        </div>
        <div class="temperature">
            <table class="temperaturetable">
                <th>Temperature details:</th>
                <td>
                    Temperature:{{temp}}
                </td>
                <td>
                    Min Temperature:{{temp_min}}
                </td>
                <td>
                    Max Temperature:{{temp_max}}
                </td>
                <td>
                    Cloud Cover:{{clouds}}
                </td>
                <td>
                    Sunrise:{{sunrise}}
                </td>
                <td>
                    Sunset:{{sunset}}
                </td>
            </table>
        </div>
        <div class="nearbyplaces">
            <div>
                <p>Near by places:</p>
                <select [(ngModel)]="selectedFilter" (change)="filteredValues()">
                    <option value="">Select a filter</option>
                    <option *ngFor="let i of placeType" [value]="i">{{i}}</option>
                </select>
            </div>
            <table class="places">
                <tr *ngFor="let item of filteredElements">
                    <th>{{ item.tags?.amenity || '' }}</th>
                    <td>{{ item.tags?.name || '' }}</td>
                    <td>{{ item.distance || '' }}</td>
                    <td><button type="button" (click)="openGoogleMaps(item.lati, item.long)">Open to Maps</button></td>
                </tr>
            </table>
        </div>
    </div>
    <div *ngIf="!mainBody && !errorMessage" class="nearbyplaces" >
        <table class="places">
            <tr *ngFor="let item of allPlaces">
                <th>{{ item.type || '' }}</th>
                <td>{{ item.name || '' }}</td>
                <td>{{ item.displayName || '' }}</td>
                <td><button type="button" (click)="getMoreDetails(item.lat,item.long)">Get details</button></td>
            </tr>
        </table>
    </div>
    <div *ngIf="!mainBody && errorMessage && allPlaces.length === 0 " class="nearbyplaces" id="errormessage">
        <p><b>No details found. Enter details properly :(</b></p>
    </div>
</div>

In [ ]:
.navbar{
    display: flex;
    justify-content: space-between;
    padding: 1%;
    background-color: #f0f2f4fa;
}

.location{
    width: 30%;
    display: flex;
}

.location button{
    margin-right: 2%;
    border: none;
    font-size: medium;
}

.location button:hover{
    cursor: pointer;
}

.form{
    width: 70%;
    display: flex;
    justify-content: end;
}

.form input{
    margin-right: 2%;
    font-size: medium;
    border-radius: 5px;
    border: 1px solid lightgray;
    padding: 5px 5px;
}

.form input:focus {
    border: 3px solid #b5b4b4;
    outline: none;
    border-radius: 6px;

}

.form button{
    font-size: medium;
    background-color: white;
    color: #707172fa;
    border-radius: 5px;
    border: 1px solid lightgray;
}

#search:hover{
    background-color: grey;
    color: white;
    cursor: pointer;
}

.mainbody{
    display: flex;
    flex-direction: column;
    padding: 10px;
    justify-content: center;
    align-items: center;
}

.locationdetails,.temperature,.nearbyplaces{
    display: flex;
    flex-direction: column;
    border-radius: 5px;
    border: 1px solid lightgray;
    margin: 5px;
    padding: 20px;
    text-align: justify;
    width: 95%;
}

.temperaturetable{
    display: flex;
    flex-direction: column;
    border-radius: 10px;
    border: 1px solid lightgray;
    overflow: hidden;
}

.temperaturetable th{
    background-color: rgb(212, 211, 211);
    border-top-left-radius: 10px;
    border-top-right-radius: 10px;
    text-align: start;
    padding: 10px;
}

.temperaturetable td{
    border: 1px solid lightgray;
    text-align: start;
    padding: 10px;
}

.locationdetails p{
    margin: 0;
    padding: 3px 0;
}

.nearbyplaces {
    display: flex;
    flex-direction: column;
    align-items: center;
    gap: 15px;
    margin-top: 20px;
}

.places {
    width: 80%;
    border-collapse: separate;
    border-spacing: 0;
}

.places tr {
    display: flex;
    flex-direction: column;
    border: 1px solid lightgray;
    border-radius: 10px;
    overflow: hidden;
    margin-bottom: 20px;
    background-color: white;
    box-shadow: 0px 2px 5px rgba(0, 0, 0, 0.1);
}

.places th {
    background-color: rgb(212, 211, 211);
    text-align: left;
    padding: 10px;
    width: auto;
}

.places td {
    padding: 5px;
    text-align: left;
    width: 100%;
}

.places td:last-child {
    border-bottom: none;
}

.places button {
    padding: 8px 12px;
    background-color: #007bff;
    color: white;
    border: none;
    cursor: pointer;
    border-radius: 5px;
    margin-top: 5px;
    align-self: flex-start;
}

.places button:hover {
    background-color: #0056b3;
}

#errormessage{
    background-color: rgb(241, 210, 174);
    color: black;
    font-size: large;
    box-shadow: 0px 2px 5px rgba(0, 0, 0, 0.3);
}

In [ ]:
import { HttpClient } from '@angular/common/http';
import { ChangeDetectorRef, Component, OnInit } from '@angular/core';
import { timestamp } from 'rxjs';

@Component({
  selector: 'app-home',
  standalone: false,
  templateUrl: './home.component.html',
  styleUrls: ['./home.component.css']
})

export class HomeComponent implements OnInit{

  latitude: number = 17.4835811;
  longitude: number = 78.3889005;
  mainBody:boolean = true;

  name:string='';
  addresstype:string='';
  displayname:string='';

  temp:string='';
  temp_min:string='';
  temp_max:string='';
  clouds:string='';
  sunrise:string='';
  sunset:string='';

  elements: ElementData[] = [];
  nearByAnemity:string='';
  nearByName:string='';

  placeType:Array<string>=[];

  selectedFilter: string = "";
  filteredElements: any[] = [];

  address:string='';
  city:string='';
  country:string='';

  allPlaces:Places[] = [];
  errorMessage:boolean=false;

  constructor(private httpclient:HttpClient,private cdr: ChangeDetectorRef){}

  ngOnInit(): void {
    this.addressDeatils(this.latitude,this.longitude);

    this.temperatureDetails(this.latitude,this.longitude);

    this.nearByPlacesDetails(this.latitude,this.longitude);

    this.errorMessage = false;
    this.mainBody = true;
  }


  addressDeatils(latit:number,longi:number){
    this.httpclient.get(`https://nominatim.openstreetmap.org/reverse?format=json&lat=${latit}&lon=${longi}`)
    .subscribe((data:any)=>{
      this.name=data.name;
      this.addresstype=data.addresstype;
      this.displayname=data.display_name;
    },
    (error)=>{
      console.error("error is ",error)
    }
    )
  }

  temperatureDetails(latit:number,longi:number){
    this.httpclient.get(`https://api.openweathermap.org/data/2.5/weather?lat=${latit}&lon=${longi}&appid=f695ee3debedce3bdbe46afca1b377f5`)
    .subscribe((data:any)=>{
      this.temp = ` ${data.main.temp} celcius`;
      this.temp_min = ` ${data.main.temp_min} celcius`;
      this.temp_max = ` ${data.main.temp_max} celcius`;
      this.clouds = ` ${data.clouds.all}%`;
      this.sunrise =` ${new Date(data.sys.sunrise*1000).toLocaleString()}`;
      this.sunset = ` ${new Date(data.sys.sunset*1000).toLocaleString()}`;
    })
  }

  nearByPlacesDetails(latit:number,longi:number){
    this.httpclient.get(`https://overpass-api.de/api/interpreter?data=[out:json];(node[%22amenity%22](around:500,${latit},${longi}););out;`)
    .subscribe((data:any)=>{
      // this.elements = data.elements;
      this.elements = data.elements.map((ele: any) => ({
        ...ele,
        distance: this.calculateDistance(this.latitude, this.longitude, ele.lat, ele.lon),
        lati: ele.lat,
        long: ele.lon
      }));
      this.filteredElements = [...this.elements];
      this.elements.forEach(ele => {
        // console.log(ele.tags?.amenity);
        if (!this.placeType.includes(ele.tags?.amenity||'')) {
          this.placeType.push(ele.tags?.amenity||'');
        }
      });
    })
  }

  calculateDistance(lat1: number, lon1: number, lat2: number, lon2: number): string {
    const R = 6371; // Radius of the Earth in km
    const dLat = (lat2 - lat1) * (Math.PI / 180);
    const dLon = (lon2 - lon1) * (Math.PI / 180);

    const a =
      Math.sin(dLat / 2) * Math.sin(dLat / 2) +
      Math.cos(lat1 * (Math.PI / 180)) * Math.cos(lat2 * (Math.PI / 180)) *
      Math.sin(dLon / 2) * Math.sin(dLon / 2);

    const c = 2 * Math.atan2(Math.sqrt(a), Math.sqrt(1 - a));
    const distance = R * c;
    return `${distance.toFixed(2)} KM`; // Distance in km
  }

  openGoogleMaps(lat: number, lon: number): void {
    console.log("Item data: ",{lat,lon});

    if (lat && lon) {
      const url = `https://www.google.com/maps?q=${lat},${lon}`;
      window.open(url, '_blank'); // Opens in a new tab
    } else {
      console.error("Latitude or Longitude is missing!");
    }
  }

  filteredValues() {
    if (!this.selectedFilter || this.selectedFilter==='Select a filter') {
        // If no filter is selected, show all rows
        this.filteredElements = [...this.elements];
        console.log(this.filteredElements);
    } else {
        // Filter elements based on selected amenity
        this.filteredElements = this.elements.filter(ele => ele.tags?.amenity === this.selectedFilter);
    }
    this.cdr.detectChanges();
  }


  getALlPlaces(){
    this.mainBody = false;
    this.errorMessage = false;
    let shopName:string = this.address.replace(" ","+");
    this.httpclient.get<any[]>(`https://nominatim.openstreetmap.org/search?q=${shopName},+${this.city},+${this.country}&format=json`)
    .subscribe((data: any[])=>{
      this.allPlaces = data.map((ele: any) => ({
        ...ele,
        type: ele.type,
        name: ele.name,
        displayName: ele.display_name,
        lat: ele.lat,
        long: ele.lon
      }));
      console.log(this.allPlaces);
    },
    (error)=>{
      this.allPlaces = [];
      this.errorMessage = true;
    }
  )
  }

  getMoreDetails(lati:number,long:number){
    this.latitude = lati;
    this.longitude = long;

    this.ngOnInit();
  }

}

interface ElementData {
  tags?:TagsData;
  lati:number;
  long:number;
  distance?:string;
}

interface TagsData{
  amenity?: string;
  name?: string;
}

interface Places{
  type:string;
  name:string;
  displayName:string;
  lat:number;
  long:number;
}

